In [ ]:
import sys
 
sys.path.append('..')
from piece_v2 import piece_v2

piece_v2.start(should_send_to_score=False)

['levels', [0, 0.4714222463226143, 0.12029133960293274, 0], 'times', [0.3, 0.5, 0.2], 'curves', [0, 0, 0]]
['levels', [0, 0.23809787871869204, 0.8343799664514153, 0], 'times', [0.2, 0.5, 0.3], 'curves', [0, 0, 0]]
['startValue', 0, 'peakValue', 0.2948034819843737]


In [185]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import *
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.instrument import AddAction, AudioInstrument
from soundmining_tools.generative import *
from soundmining_tools.sequencer import SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from piece_v2 import *
from soundmining_tools.spectrum import make_fm_synthesis, make_fact
from soundmining_tools.note import note_to_hertz
from enum import StrEnum
from dataclasses import dataclass
from typing import Optional

piece_v2.reset()
piece_v2.synth_player.start()

static_control = piece_v2.instruments.static_control
sine_control = piece_v2.instruments.sine_control
perc_control = piece_v2.instruments.perc_control
line_control = piece_v2.instruments.line_control
signal_sum = piece_v2.instruments.signal_sum
signal_multiply = piece_v2.instruments.signal_multiply
three_block_control = piece_v2.instruments.three_block_control

fundamental = note_to_hertz("d2")
first_partial = note_to_hertz("d3")

base_spectrum = make_fm_synthesis(fundamental, first_partial, 50)

mod_ratio1 = make_fact(base_spectrum[6][0], base_spectrum[9][0])
mod_ratio2 = make_fact(base_spectrum[7][0], base_spectrum[8][0])

fm_spectrum = make_fm_synthesis(fundamental, fundamental * mod_ratio1, 50)
spectrum = [n[0] for n in fm_spectrum]
sub_spectrum = [n[1] for n in fm_spectrum]

print(fundamental, first_partial, mod_ratio1)
print(spectrum)
print(sub_spectrum)

Pitch = StrEnum("Pitch", ["LOW", "MIDDLE_LOW", "MIDDLE_HIGH", "HIGH"])
SpectrumType = StrEnum("SpectrumType", ["NORMAL", "SUB"])
Time = StrEnum("Time", ["SHORT", "MIDDLE", "LONG", "SHORT_PAUSE", "LONG_PAUSE"])
GroupType = StrEnum("GroupType", ["PAD", "LOW"])

@dataclass
class GroupNote:
    primary: tuple[GroupType, Time]
    secondary: Optional[tuple[Time, GroupType]] = None

pad_pitch_chain = MarkovChain({
    Pitch.MIDDLE_LOW: {Pitch.MIDDLE_LOW: 0.0, Pitch.MIDDLE_HIGH: 0.5, Pitch.HIGH: 0.5},
    Pitch.MIDDLE_HIGH: {Pitch.MIDDLE_LOW: 0.2, Pitch.MIDDLE_HIGH: 0.4, Pitch.HIGH: 0.4},
    Pitch.HIGH: {Pitch.MIDDLE_LOW: 0.2, Pitch.MIDDLE_HIGH: 0.4, Pitch.HIGH: 0.4}
}, Pitch.MIDDLE_HIGH)

pad_spectrum_type_chain = MarkovChain({
    SpectrumType.NORMAL: {SpectrumType.NORMAL: 0.5, SpectrumType.SUB: 0.5},
    SpectrumType.SUB: {SpectrumType.NORMAL: 0.5, SpectrumType.SUB: 0.5}
}, SpectrumType.NORMAL)

low_pitch_chain = MarkovChain({
    Pitch.LOW: {Pitch.LOW: 0.5, Pitch.MIDDLE_LOW: 0.5},
    Pitch.MIDDLE_LOW: {Pitch.LOW: 0.8, Pitch.MIDDLE_LOW: 0.2},
}, Pitch.LOW)

low_spectrum_type_chain = MarkovChain({
    SpectrumType.NORMAL: {SpectrumType.NORMAL: 0.5, SpectrumType.SUB: 0.5},
    SpectrumType.SUB: {SpectrumType.NORMAL: 0.5, SpectrumType.SUB: 0.5}
}, SpectrumType.NORMAL)

high_pan_points = [(-0.99, -0.75), (0.75, 0.99)]
middle_pan_points = [(-0.66, -0.33), (0.33, 0.66)]
low_pan_points = [(-0.25, 0), (0, 0.25)]

class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient):
        super().__init__(client)

    
    def low_modindex(self, amp: float) -> float:
        return 10 + (random_range(5, 20) * amp)
    
    def high_modindex(self, amp: float) -> float:
        return 100 + (random_range(500, 2000) * amp)


    def variant2_note(self, start: float, pitch: float, amp: float, amp_control: AudioInstrument, duration: float, pan_control: AudioInstrument) -> None:
        mod_freq1 = static_control(pitch * mod_ratio1)        
        mod_amp1 = static_control(self.high_modindex(amp))                
        mod1 = (
            piece_v2.synth_player.note()
            .sine(freq=mod_freq1, amp=mod_amp1)
            .audio_stack.pop()
        )

        mod_freq2 = static_control(pitch * mod_ratio2)      
        mod_index2 = static_control(self.low_modindex(amp))     # Turn to noise with larger values        
        mod_amp2 = signal_multiply(mod_freq2, mod_index2).add_action(AddAction.TAIL_ACTION)
        fm_mod2 = signal_sum(mod_freq2, mod1).add_action(AddAction.TAIL_ACTION)
        mod2 = (
            piece_v2.synth_player.note()
            .sine(freq=fm_mod2, amp=mod_amp2)
            .audio_stack.pop()
        )

        car_freq = static_control(pitch)
        car_amp = amp_control
        fm_mod = signal_sum(car_freq, mod2).add_action(AddAction.TAIL_ACTION)
        (
            piece_v2.synth_player.note()
                .sine(freq=fm_mod, amp=car_amp)
                .pan(pan_control)
                .play(start, duration)
        )


    def variant1_note(self, start: float, pitch: float, amp: float, amp_control: AudioInstrument, duration: float, pan_control: AudioInstrument) -> None:
        mod_freq = static_control(pitch * mod_ratio1)        
        mod_amp = static_control(self.high_modindex(amp))        
        mod = (
            piece_v2.synth_player.note()
            .sine(freq=mod_freq, amp=mod_amp)
            .audio_stack.pop()
        )        

        car_freq = static_control(pitch)
        car_amp = amp_control
        fm_mod = signal_sum(car_freq, mod).add_action(AddAction.TAIL_ACTION)
        (
            piece_v2.synth_player.note()
                .sine(freq=fm_mod, amp=car_amp)
                .pan(pan_control)
                .play(start, duration)
        )

    def play_pad_note(self, start: float, amp: float, pitch: float, pan: float) -> SequenceNote:
        amp_control = sine_control(0, amp)
        pan_control = static_control(pan)
        duration = random_range(8, 13)
        self.variant1_note(start=start, pitch=pitch, amp=amp, amp_control=amp_control, duration=duration, pan_control=pan_control)
        return SequenceNote(start=start, track="PAD", duration=duration, freq=pitch)


    def play_low_note(self, start: float, amp: float, pitch: float, pan: float) -> SequenceNote:
        #amp_control = random.choice([
        #    sine_control(0, amp),
        #    three_block_control((0, amp / random_range(3, 5), amp, 0), (0.2, 0.5, 0.3), (0, 0, 0)),
        #    three_block_control((0, amp, amp / random_range(3, 5), 0), (0.3, 0.5, 0.2), (0, 0, 0))
        #])

        amp_control = sine_control(0, amp)
        pan_control = static_control(pan)
        duration = random_range(5, 8)
        self.variant2_note(start=start, pitch=pitch, amp=amp, amp_control=amp_control, duration=duration, pan_control=pan_control)
        return SequenceNote(start=start, track="LOW", duration=duration, freq=pitch)

    def play_pad_group(self, start: float) -> list[SequenceNote]:
        time = start
        notes = []
        for _ in range(random_int_range(3, 5)):
            match pad_spectrum_type_chain.next():
                case SpectrumType.NORMAL:
                    sp = spectrum
                case SpectrumType.SUB:
                    sp = sub_spectrum

            match pad_pitch_chain.next():
                case Pitch.MIDDLE_LOW:
                    pitch_index = random_int_range(10, 19)
                    pan = pan_point(middle_pan_points)
                case Pitch.MIDDLE_HIGH:
                    pitch_index = random_int_range(20, 29)
                    pan = pan_point(high_pan_points)
                case Pitch.HIGH:
                    pitch_index = random_int_range(30, 39)
                    pan = pan_point(high_pan_points)
            
            pitch = sp[pitch_index]
            amp = random_range(0.15, 0.85)
            notes.append(self.play_pad_note(time, amp, pitch, pan))
            time += random_range(3, 5)
        return notes

    def play_low_group (self, start: float) -> list[SequenceNote]:
        time = start
        notes = []
        for _ in range(random_int_range(2, 3)):
            match low_spectrum_type_chain.next():
                case SpectrumType.NORMAL:
                    sp = spectrum
                case SpectrumType.SUB:
                    sp = sub_spectrum

            match low_pitch_chain.next():
                case Pitch.LOW:
                    pitch_index = random_int_range(0, 9)
                    pan = pan_point(low_pan_points)
                case Pitch.MIDDLE_LOW:
                    pitch_index = random_int_range(10, 19)                
                    pan = pan_point(middle_pan_points)
            
            pitch = sp[pitch_index]
            amp = random_range(0.15, 0.85)
            notes.append(self.play_low_note(time, amp, pitch, pan))
            time += random_range(1, 2)
        return notes

    def get_notes_duration(self, start: float, notes: list[SequenceNote]) -> float:
        max_duration = 0
        for note in notes:
            relative_duration = (note.start - start) + note.duration
            max_duration = max(max_duration, relative_duration)        
        return max_duration


    piece_sequence = [
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),

        GroupNote((GroupType.LOW, Time.LONG_PAUSE), (Time.SHORT, GroupType.PAD)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG)),
                
        GroupNote((GroupType.PAD, Time.LONG)),

        GroupNote((GroupType.LOW, Time.SHORT_PAUSE)),
        GroupNote((GroupType.LOW, Time.SHORT_PAUSE)),
        GroupNote((GroupType.LOW, Time.SHORT_PAUSE), (Time.MIDDLE, GroupType.PAD)),
        GroupNote((GroupType.LOW, Time.SHORT_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG)),

        GroupNote((GroupType.PAD, Time.SHORT_PAUSE)),
        GroupNote((GroupType.PAD, Time.SHORT_PAUSE)),
        GroupNote((GroupType.PAD, Time.SHORT_PAUSE)),
        GroupNote((GroupType.PAD, Time.SHORT_PAUSE), (Time.MIDDLE, GroupType.LOW)),
        GroupNote((GroupType.PAD, Time.LONG)),

        GroupNote((GroupType.LOW, Time.LONG_PAUSE)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE), (Time.LONG, GroupType.PAD)),
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)), 
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)), #?
        GroupNote((GroupType.LOW, Time.LONG_PAUSE)), #?  
    ]

    def play_piece(self, start: float) -> list[SequenceNote]:
        time = start
        all_notes = []
        for group_note in self.piece_sequence:
            group_type, time_duration  = group_note.primary
            match group_type:
                case GroupType.LOW:
                    notes = self.play_low_group(time)
                    all_notes.extend(notes)
                case GroupType.PAD:
                    notes = self.play_pad_group(time)
                    all_notes.extend(notes)
            group_duration = self.get_notes_duration(time, notes)
            if group_note.secondary:
                secondary_time_duration, secondary_group_type = group_note.secondary
                match secondary_time_duration:
                    case Time.SHORT:
                        delay = group_duration * random_range(0.15, 0.33)
                    case Time.MIDDLE:
                        delay = group_duration * random_range(0.33, 0.66)
                    case Time.LONG:
                        delay = group_duration * random_range(0.66, 0.85)
                    case Time.SHORT_PAUSE:
                        delay = group_duration + random_range(1, 2)
                    case Time.LONG_PAUSE:
                        delay = group_duration + random_range(3, 5)
                match secondary_group_type:
                    case GroupType.LOW:
                        notes = self.play_low_group(time + delay)
                        all_notes.extend(notes)
                    case GroupType.PAD:
                        notes = self.play_pad_group(time + delay)
                        all_notes.extend(notes)
            match time_duration:
                case Time.SHORT:
                    time += group_duration * random_range(0.15, 0.33)
                case Time.MIDDLE:
                    time += group_duration * random_range(0.33, 0.66)
                case Time.LONG:
                    time += group_duration * random_range(0.66, 0.85)
                case Time.SHORT_PAUSE:
                    time += group_duration + random_range(1, 2)
                case Time.LONG_PAUSE:
                    time += group_duration + random_range(3, 5)
        return all_notes
            
    def display_stats(self, notes: list[SequenceNote]):
        ui_piece = UiPieceBuilder().add_notes(notes).build()
        piece_duration = ui_piece.get_duration()
        piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

        min_freq = 0
        max_freq = 0
        for track in ui_piece.tracks:
            track_duration = 0
            for note in track.notes:
                track_duration = max(track_duration, note.start + note.duration)
                min_freq = min(min_freq, note.freq)
                max_freq = max(max_freq, note.freq)

            piece_stats[track.track_name] = track_duration
        display(piece_stats)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.octave:
            case 2:
                self.play_low_group(patch_arguments.start)
            case 3: 
                self.play_pad_group(patch_arguments.start)  
            case 4:
                notes = self.play_piece(patch_arguments.start)
                self.display_stats(notes)
        
            

my_handler = MyHandler(piece_v2.supercollider_client)
piece_v2.receiver.set_note_handler(my_handler)


73.41620171654216 146.83240343308432 0.4615384615384617
[73.41620171654216, 107.3006025087924, 141.18500330104263, 175.0694040932929, 208.95380488554312, 242.83820567779335, 276.7226064700436, 310.60700726229385, 344.4914080545441, 378.3758088467943, 412.26020963904455, 446.1446104312948, 480.02901122354507, 513.9134120157953, 547.7978128080456, 581.6822136002958, 615.566614392546, 649.4510151847962, 683.3354159770465, 717.2198167692968, 751.1042175615469, 784.9886183537972, 818.8730191460475, 852.7574199382977, 886.641820730548, 920.5262215227981, 954.4106223150484, 988.2950231072987, 1022.179423899549, 1056.063824691799, 1089.9482254840493, 1123.8326262762996, 1157.71702706855, 1191.6014278608002, 1225.4858286530505, 1259.3702294453005, 1293.2546302375508, 1327.1390310298011, 1361.0234318220514, 1394.9078326143015, 1428.7922334065518, 1462.676634198802, 1496.5610349910523, 1530.4454357833026, 1564.3298365755527, 1598.214237367803, 1632.0986381600533, 1665.9830389523036, 1699.86743974

{'total': 370.82384446970354,
 'total minutes': 6.1803974078283925,
 'tracks': 2,
 'LOW': 370.82384446970354,
 'PAD': 350.06244894935435}